# Writer Tools


This notebook provides a quick overview for getting started with Writer [tools](https://python.langchain.com/docs/concepts/tools/). For detailed documentation of all Writer features and configurations head to the [Writer docs](https://dev.writer.com/home).

## Overview

### Integration details

| Class                                                                                                      | Package          | Local | Serializable | JS support |                                        Package downloads                                         |                                        Package latest                                         |
|:-----------------------------------------------------------------------------------------------------------|:-----------------| :---: | :---: |:----------:|:------------------------------------------------------------------------------------------------:|:---------------------------------------------------------------------------------------------:|
| [GraphTool](https://github.com/writer/langchain-writer/blob/main/langchain_writer/tools.py#L9) | [langchain-writer](https://pypi.org/project/langchain-writer/) |      ❌       |                                       ❌                                       | ❌ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-writer?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-writer?style=flat-square&label=%20) |

### Features

We provide usage of four types of tools for use with `ChatWriter`: `function`, `llm`, `graph` and `application`.

#### Function

Functions are the most common type of tool, which allows the LLM to call external APIs, fetch data from databases, and generally perform any external action you want to do. Visit our [tool calling docs](https://dev.writer.com/api-guides/tool-calling#tool-calling) for additional information.

#### Graph

The `Graph` tool is Writer's graph-based retrieval-augmented generation (RAG) called Knowledge Graph. This tool enables developers to simply pass the graph ID to the model, and it will return the answer to the question in the prompt. To learn more, see our [Knowledge Graph API docs](https://dev.writer.com/api-guides/knowledge-graph).

#### LLM

The `LLM` tool is a Writer specific tool, which allows performing sub calls to different Palmyra model types. As well as `Graph` tool `LLM` tool is a remotely executing tool, which means you don't have to worry about its execution: Writer server will do it for you. To learn more, see our [LLM tool docs](https://dev.writer.com/api-guides/model-delegation#tool-object).

#### No-code app

The `No-code app` tool is a wrapper for Writer [no-code apps](https://dev.writer.com/api-guides/applications#no-code-applications) to use them as tools for LLM. In distinguish from `Graph` and `LLM` tools `No-code app` tool requires manual execution and passing tool call results into messages history, because it's based on usual `function` tool type. For further information see our [No-code apps as tool docs](https://dev.writer.com/api-guides/applications-tool-calling#no-code-applications-as-tools).

## Setup



### Installation

In [ ]:
%pip install -qU langchain-writer

### Credentials

Sign up for [Writer AI Studio](https://app.writer.com/aistudio/signup?utm_campaign=devrel) to generate an API key (you can follow this [Quickstart](https://dev.writer.com/api-guides/quickstart)). Then, set the WRITER_API_KEY environment variable:

In [ ]:
import getpass
import os

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

## Usage

You can bind graph or function tools to `ChatWriter`.

### Graph Tools

To bind graph tools, first create and initialize a `GraphTool` instance with the `graph_ids` you want to use as sources:

In [ ]:
from langchain_writer.chat_models import ChatWriter
from langchain_writer.tools import GraphTool

chat = ChatWriter()

graph_id = getpass.getpass("Enter Writer Knowledge Graph ID: ")
graph_tool = GraphTool(graph_ids=[graph_id])

### LLM Tools

To bind LLM tools, first create and initialize a `LLMTool` instance with the `model_name` you want to use as sources:

In [ ]:
from langchain_writer.tools import LLMTool

llm_tool = LLMTool(model_name="palmyra-med")

### No-code app Tools

To bind no-code app tools, first create and initialize a `NoCodeAppTool` instance with the `app_id` you want to use as sources:

In [ ]:
from langchain_writer.tools import NoCodeAppTool

app_id = getpass.getpass("Enter App ID: ")
app_tool = NoCodeAppTool(app_id=app_id)

## Instantiation

In [ ]:
from typing import Optional

from langchain_core.tools import tool
from pydantic import BaseModel, Field


@tool
def get_supercopa_trophies_count(club_name: str) -> Optional[int]:
    """Returns information about supercopa trophies count.

    Args:
        club_name: Club you want to investigate info of supercopa trophies about

    Returns:
        Number of supercopa trophies or None if there is no info about requested club
    """

    if club_name == "Barcelona":
        return 15
    elif club_name == "Real Madrid":
        return 13
    elif club_name == "Atletico Madrid":
        return 2
    else:
        return None


class GetWeather(BaseModel):
    """Get the current weather in a given location"""

    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")


get_product_info = {
    "type": "function",
    "function": {
        "name": "get_product_info",
        "description": "Get information about a product by its id",
        "parameters": {
            "type": "object",
            "properties": {
                "product_id": {
                    "type": "number",
                    "description": "The unique identifier of the product to retrieve information for",
                }
            },
            "required": ["product_id"],
        },
    },
}

### Binding tools
Then, you can simply bind all tools to the `ChatWriter` instance:

In [ ]:
chat_with_tools = chat.bind_tools(
    [graph_tool, llm_tool, app_tool, get_supercopa_trophies_count, GetWeather, get_product_info]
)

## Invocation

The model will automatically choose the tool during invocation with all modes (streaming/non-streaming, sync/async).

In [ ]:
from langchain_core.messages import HumanMessage

messages = [
    HumanMessage(
        "Use knowledge graph tool to compose this answer. Tell me what th first line of documents stored in your KG. Also I want to know: how many SuperCopa trophies have Barcelona won?"
    )
]

response = chat_with_tools.invoke(messages)
messages.append(response)

In the case of function tools, you will receive an assistant message with the tool call request.

In [ ]:
print(response.tool_calls)

Then you can manually handle tool call request, send to model and receive final response:

In [ ]:
for tool_call in response.tool_calls:
    selected_tool = {
        "get_supercopa_trophies_count": get_supercopa_trophies_count,
    }[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

response = chat_with_tools.invoke(messages)
print(response.content)

With a `GraphTool` and `LLMTool`, the model will call it remotely and return usage info in the `additional_kwargs` under the `graph_data` and `llm_data` keys respectively:

In [ ]:
print(response.additional_kwargs["graph_data"])
print(response.additional_kwargs["llm_data"])

The `content` attribute contains the final response:

In [ ]:
print(response.content)

## Chaining

Due to specificity of Writer Graph and LLM tools (you don't need to call it manually, Writer server will call it by himself and return RAG/sub LLM call based generation) it's impossible to invoke them separately, so `GraphTool` and `LLMTool` can't be used as part of chain.